# Multiprocessing: exec; inter-process communication
_COSC 208, Introduction to Computer Systems, 2023-11-08_

## Announcements
* Project 3 due tomorrow @ 11pm

## Outline
* Warm-up
* Running a different program
* Interprocess communication

## Warm-up

* Q1: _What are all possible outputs of this program, assuming the process identifier (PID) of the parent process is 50 and the PID of the child process is 60?_

In [ ]:
#include <stdio.h>
#include <unistd.h>    
int main() {
    int n = 20;
    int x = fork();
    if (x < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (x == 0) {
        n += 5;
    } else {
        wait(NULL);
        n += 10;
    }
    printf("x=%d n=%d\n", x, n);
}

    x=0 n=25
    x=60 n=30

<div style="height:15em;"></div>

🛑 **STOP here** after completing the above question; if you have extra time **take a few deep breaths** to reduce stress.

<div style="page-break-after:always;"></div>

## Running a different program

* `int exec(const char *path, const char *argv[])`
    * Used to switch which program is running in a process — replaces current code with code for a new program and starts executing that program from main
    * `path` == full path to program
    * `argv` == array of strings containing the full path to the program, any command line arguments, and `NULL`

* Example program

In [5]:
#include <stdio.h>
#include <unistd.h>
int main(int argc, char **argv) {
    printf("Begin\n");
    int pid = fork();
    if (pid == 0) {
        printf("Child\n");
        char *cmd[] = { "/bin/date", NULL };
        execv(cmd[0], cmd);
    } else {
        printf("Parent\n");
    }
    printf("End\n");
    return 0;
}

Begin
Parent
End
Child
Sun Nov  5 20:18:17 EST 2023


* Q2: _What is the output produced by running `./progA`, assuming no errors occur?_

In [ ]:
/** progA **/
#include <stdio.h>
#include <unistd.h>
int main() {
    pid_t a = fork();
    if (a == 0) {
        char *cmd[] = { "./progB", NULL };
        execv(cmd[0], cmd);
        printf("A 2nd gen\n");
        return 0;
    } else {
        wait(NULL);
        printf("A 1st gen\n");
        return 0;
    }
}

In [ ]:
/** progB **/
#include <stdio.h>
#include <unistd.h>
int main() {
    pid_t b = fork();
    if (b == 0) {
        printf("B 2nd gen\n");
        return 0;
    } else {
        wait(NULL);
        printf("B 1st gen\n");
        return 0;
    }
}

    B 2nd gen
    B 1st gen
    A 1st gen

<div style="page-break-after:always;"></div>

## Interprocess communication

* Signals
    * Send using the `kill` system call
    * Register a signal handler using the `signal` system call – must register before calling fork

* Example with signals

In [ ]:
#include <signal.h>
#include <stdio.h>
#include <unistd.h>

void sigusr1_handler(int sig) {
    printf("Received %d\n", sig);
    fflush(stdout);
}

int main() {
    if (signal(SIGURG, sigusr1_handler) == SIG_ERR) {
        return 1;
    }
    
    int result = fork();
    if (result < 0) {
        return 1;
    } else if (result == 0) {
        sleep(1);
        printf("Child\n");
        fflush(stdout);
    } else {
        if (kill(result, SIGURG) < 0) {
            return 1;
        }
        wait(NULL);
        printf("Parent\n");
    }
    return 0;
}

* Message passing
    * pipe – one-way communcation channel for two processes on the same machine
    * socket - dual-way communication channel for two processes on the same or different machines

<div style="page-break-after:always;"></div>

* Example with pipes

In [ ]:
#include <signal.h>
#include <stdio.h>
#include <unistd.h>

int main() {
    int pipe_descriptors[2];
    if (pipe(pipe_descriptors) < 0) {
        return 1;
    }

    int result = fork();
    if (result < 0) {
        return 1;
    } else if (result == 0) {
        printf("Child\n");
        char msg[1024];
        read(pipe_descriptors[0], msg, 1024);
        printf("Read: %s\n", msg);
    } else {
        char msg[1024] = "Hello from your parent!";
        write(pipe_descriptors[1], msg, 1024);
        wait(NULL);
        printf("Parent\n");
    }
    return 0;
}